# Use LangChain with Meta Llama 3.1 - 8b Instruct NIM in Azure AI Foundry and Azure ML

You can use Meta-Llama-3 models family of models deployed in Azure AI Foundry and Azure ML with `langchain` to create more sophisticated intelligent applications. Use `langchain_community` package with the Azure Machine Learning integration.

## Prerequisites

Before we start, there are certain steps we need to take to deploy the models:

* Register for a valid Azure account with subscription 
* Make sure you have access to [Azure AI Studio](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio?tabs=home)
* Create a project and resource group
* Select Nvidia NIM:  Meta Llama 3.1 -8b Instruct NIM models from Model catalog


![nim-models.png](nim-models.png)

Once deployed successfully, you should be assigned for an API endpoint and a security key for inference. 



To complete this tutorial, you will need to:

* Install `langchain` and `langchain_community`:

    ```bash
    pip install langchain langchain_community
    ```

## Example

The following example demonstrate how to create a chain that uses a Meta-Llama-3 chat model deployed in Azure AI and Azure ML. The chain has been configured with a `ConversationBufferMemory`. This example has been adapted from [LangChain official documentation](https://python.langchain.com/docs/modules/memory/adding_memory).

In [ ]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage
from langchain_community.chat_models.azureml_endpoint import (
    AzureMLChatOnlineEndpoint,
    AzureMLEndpointApiType,
    LlamaChatContentFormatter,
)

Let's create an instance of our `AzureMLChatOnlineEndpoint` model. This class allow us to get access to any model deployed in Azure AI or Azure ML. For completion models use class `langchain_community.llms.azureml_endpoint.AzureMLOnlineEndpoint` with `LlamaContentFormatter` as the `content_formatter`.

In [ ]:
chat_model = AzureMLChatOnlineEndpoint(
    # endpoint_url="https://<endpoint-name>.<region>.inference.ai.azure.com/v1/chat/completions",
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    # endpoint_api_key="<key>",
    content_formatter=LlamaChatContentFormatter(),
)

> Tip: You can configure environment variables `AZUREML_ENDPOINT_URL`, `AZUREML_ENDPOINT_API_KEY`, and `AZUREML_ENDPOINT_API_TYPE` instead of passing them as arguments.

In the below prompt, we have two input keys: one for the actual input (`human_input`), and another for the input from the `Memory` class (`chat_history`).

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a chatbot having a conversation with a human. You love making references to animals on your answers."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{human_input}"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

We create the chain as follows:

In [ ]:
chat_llm_chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    memory=memory,
    verbose=True,
)

We can see how it works:

In [ ]:
chat_llm_chain.predict(human_input="Hi there my friend")

In [ ]:
chat_llm_chain.predict(
    human_input="I'm thinking on a present for my mother. Any advise?"
)